In [1]:
#Importing required libraries 

# Keras
import keras
from keras import regularizers
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.utils import np_utils, to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import load_model, model_from_json

# sklearn
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Other  
import librosa
import librosa.display
import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import pandas as pd
import seaborn as sns
import glob 
import os
import pickle
import IPython.display as ipd  # To play audio in notebook
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
print(tf.__version__)

2.2.0


# Data Preparation

In [3]:
# load csv file created in extracting labels and path 
df = pd.read_csv("All_Data_Frames.csv")
df.head()

,labels,source,path
0,male_neutral,SAVEE,/home/bukya/Desktop/Speech_Emotion_Recognition...
1,male_fear,SAVEE,/home/bukya/Desktop/Speech_Emotion_Recognition...
2,male_surprise,SAVEE,/home/bukya/Desktop/Speech_Emotion_Recognition...
3,male_sad,SAVEE,/home/bukya/Desktop/Speech_Emotion_Recognition...
4,male_disgust,SAVEE,/home/bukya/Desktop/Speech_Emotion_Recognition...


In [4]:
df.shape

(12162, 3)

In [5]:
df.path[12050], df.labels[12050]

('/home/bukya/Desktop/Speech_Emotion_Recognition/AudioWAV/1090_MTI_ANG_XX.wav',
 'male_angry')

# Merge labels

In [6]:
# merge labels 
for i in range(len(df)):
    if 'angry' in df.labels[i]:
        df.labels[i] = 'Anger'
    elif 'disgust' in df.labels[i]:
        df.labels[i] = '-'
    elif 'fear' in df.labels[i]:
        df.labels[i] = 'Fear'
    elif 'happy' in df.labels[i]:
        df.labels[i] = 'Joy'
    elif 'neutral' in df.labels[i]:
        df.labels[i] = 'None'
    elif 'sad' in df.labels[i]:
        df.labels[i] = 'Sad'
    elif 'surprise' in df.labels[i]:
        df.labels[i] = '-'

In [7]:
df.labels.value_counts()

-        2575
Joy      1923
Sad      1923
Anger    1923
Fear     1923
None     1895
Name: labels, dtype: int64

In [8]:
df.head(10)

,labels,source,path
0,None,SAVEE,/home/bukya/Desktop/Speech_Emotion_Recognition...
1,Fear,SAVEE,/home/bukya/Desktop/Speech_Emotion_Recognition...
2,-,SAVEE,/home/bukya/Desktop/Speech_Emotion_Recognition...
3,Sad,SAVEE,/home/bukya/Desktop/Speech_Emotion_Recognition...
4,-,SAVEE,/home/bukya/Desktop/Speech_Emotion_Recognition...
5,Joy,SAVEE,/home/bukya/Desktop/Speech_Emotion_Recognition...
6,Sad,SAVEE,/home/bukya/Desktop/Speech_Emotion_Recognition...
7,-,SAVEE,/home/bukya/Desktop/Speech_Emotion_Recognition...
8,-,SAVEE,/home/bukya/Desktop/Speech_Emotion_Recognition...
9,-,SAVEE,/home/bukya/Desktop/Speech_Emotion_Recognition...


In [9]:
# drop surprise, disgust labels
df1 = df.drop(df[(df.labels == '-')].index).reset_index(drop=True)
df1.head()

,labels,source,path
0,None,SAVEE,/home/bukya/Desktop/Speech_Emotion_Recognition...
1,Fear,SAVEE,/home/bukya/Desktop/Speech_Emotion_Recognition...
2,Sad,SAVEE,/home/bukya/Desktop/Speech_Emotion_Recognition...
3,Joy,SAVEE,/home/bukya/Desktop/Speech_Emotion_Recognition...
4,Sad,SAVEE,/home/bukya/Desktop/Speech_Emotion_Recognition...


In [10]:
df1.shape

(9587, 3)

In [11]:
df2 = df1.sample(frac=1, random_state=42).reset_index(drop=True) # shuffle dataframe 

In [14]:
df2.tail()

,labels,source,path
9582,Sad,CREMA,/home/bukya/Desktop/Speech_Emotion_Recognition...
9583,Anger,CREMA,/home/bukya/Desktop/Speech_Emotion_Recognition...
9584,Anger,CREMA,/home/bukya/Desktop/Speech_Emotion_Recognition...
9585,Sad,RAVDESS,/home/bukya/Desktop/Speech_Emotion_Recognition...
9586,Sad,CREMA,/home/bukya/Desktop/Speech_Emotion_Recognition...


In [30]:
# split the dataset 
train_df, test_df = train_test_split(df2, test_size=0.05, random_state=6, stratify=df2.labels)

In [31]:
# restet the index
train_df1 = train_df.reset_index(drop=True)
test_df1 = test_df.reset_index(drop=True)

In [32]:
train_df1.shape, test_df1.shape

((9107, 3), (480, 3))

In [33]:
train_df1.tail(), test_df1.tail()

(     labels   source                                               path
 9102    Sad     TESS  /home/bukya/Desktop/Speech_Emotion_Recognition...
 9103    Sad    CREMA  /home/bukya/Desktop/Speech_Emotion_Recognition...
 9104   Fear  RAVDESS  /home/bukya/Desktop/Speech_Emotion_Recognition...
 9105  Anger    CREMA  /home/bukya/Desktop/Speech_Emotion_Recognition...
 9106    Joy    CREMA  /home/bukya/Desktop/Speech_Emotion_Recognition...,
     labels   source                                               path
 475   None    CREMA  /home/bukya/Desktop/Speech_Emotion_Recognition...
 476   None     TESS  /home/bukya/Desktop/Speech_Emotion_Recognition...
 477    Joy     TESS  /home/bukya/Desktop/Speech_Emotion_Recognition...
 478   Fear  RAVDESS  /home/bukya/Desktop/Speech_Emotion_Recognition...
 479   None    CREMA  /home/bukya/Desktop/Speech_Emotion_Recognition...)

In [34]:
train_df1.labels.value_counts(), test_df1.labels.value_counts()

(Anger    1827
 Fear     1827
 Sad      1827
 Joy      1826
 None     1800
 Name: labels, dtype: int64, Joy      97
 Sad      96
 Anger    96
 Fear     96
 None     95
 Name: labels, dtype: int64)

# Feature Extraction

In [48]:
# helper function for feature extraction
# using librosa library 
def feature_extraction(dataframe):
    mfcc_features = [] # fetures
    classes = [] # labels
    file_path = [] # file path
    for i in tqdm(range(len(dataframe))):
        file = dataframe.path[i]
        label = dataframe.labels[i]
        y, sr = librosa.load(file) # load audio file
        mfccs = np.mean(librosa.feature.mfcc(y, sr, n_mfcc=40).T,axis=0) # 40 used for previous model
        mfcc_features.append(mfccs) # append extracted features
        classes.append(label) # append respective class
        file_path.append(file) # append file path for prediction analysis
    return mfcc_features, classes, file_path 

In [49]:
# extract features using above helper function
x_train, y_train, train_file_path = feature_extraction(train_df1)
x_test, y_test, test_file_path = feature_extraction(test_df1)


  3%|▎         | 303/9107 [00:57<25:38,  5.72it/s]


  7%|▋         | 605/9107 [02:01<27:16,  5.20it/s]


 10%|▉         | 907/9107 [03:04<32:48,  4.17it/s]


 13%|█▎        | 1205/9107 [03:59<20:45,  6.35it/s]


 16%|█▋        | 1501/9107 [04:57<29:50,  4.25it/s]


...

 94%|█████████▍| 8606/9107 [27:39<01:30,  5.55it/s]


 98%|█████████▊| 8902/9107 [28:29<00:33,  6.10it/s]


100%|██████████| 9107/9107 [29:05<00:00,  5.22it/s]

 20%|██        | 96/480 [00:16<01:01,  6.27it/s]


 84%|████████▍ | 404/480 [01:10<00:12,  6.02it/s]


100%|██████████| 480/480 [01:23<00:00,  5.76it/s]


In [50]:
len(x_train[0]), len(x_test[0])

(100, 100)

In [51]:
# save test set for tflite model performance monitoring
test_df = pd.DataFrame(list(zip(test_file_path, y_test)), columns=['path', 'label'])
test_df.head()

,path,label
0,/home/bukya/Desktop/Speech_Emotion_Recognition...,Joy
1,/home/bukya/Desktop/Speech_Emotion_Recognition...,Joy
2,/home/bukya/Desktop/Speech_Emotion_Recognition...,Fear
3,/home/bukya/Desktop/Speech_Emotion_Recognition...,Anger
4,/home/bukya/Desktop/Speech_Emotion_Recognition...,Anger


In [52]:
test_df.tail()

,path,label
475,/home/bukya/Desktop/Speech_Emotion_Recognition...,None
476,/home/bukya/Desktop/Speech_Emotion_Recognition...,None
477,/home/bukya/Desktop/Speech_Emotion_Recognition...,Joy
478,/home/bukya/Desktop/Speech_Emotion_Recognition...,Fear
479,/home/bukya/Desktop/Speech_Emotion_Recognition...,None


In [53]:
test_df.shape

(480, 2)

In [54]:
# save test for predictions
test_df.to_csv('test_data_5labels_97.csv', index=False)

In [55]:
#converting the lists into numpy arrays
x_train=np.array(x_train)
x_test=np.array(x_test)
y_train=np.array(y_train)
y_test=np.array(y_test)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((9107, 100), (480, 100), (9107,), (480,))

In [56]:
# one hot encode the target 
lb = LabelEncoder()
y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))

In [57]:
# number of classes
len(y_train[0]), len(y_test[0])

(5, 5)

In [58]:
print(x_train.shape)
print(lb.classes_)
#print(y_train[0:10])
#print(y_test[0:10])

# Pickel the lb object for future use 
filename = 'labels.txt'
outfile = open(filename,'wb')
pickle.dump(lb.classes_,outfile)
outfile.close()

(9107, 100)
['Anger' 'Fear' 'Joy' 'None' 'Sad']


In [59]:
#reshaping to shape required by CNN
x_train=np.reshape(x_train,(x_train.shape[0], 40,1,1))
x_test=np.reshape(x_test,(x_test.shape[0], 40,1,1))

In [60]:
#shapes
x_train.shape,x_test.shape

((9107, 100, 1, 1), (480, 100, 1, 1))

In [61]:
from tensorflow.keras import Sequential
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras import Model

2.2.0


In [62]:
#forming model
model=Sequential()

In [63]:
#adding layers and forming the model
model.add(Conv2D(64,kernel_size=5,strides=1,padding="Same",activation="relu",input_shape=(40,1,1)))
model.add(MaxPooling2D(padding="same"))

model.add(Conv2D(128,kernel_size=5,strides=1,padding="same",activation="relu"))
model.add(MaxPooling2D(padding="same"))
model.add(Dropout(0.3))

model.add(Flatten())

model.add(Dense(256,activation="relu"))
model.add(Dropout(0.3))

model.add(Dense(512,activation="relu"))
model.add(Dropout(0.3))

model.add(Dense(5,activation="softmax"))

In [64]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 100, 1, 64)        1664      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 50, 1, 64)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 50, 1, 128)        204928    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 1, 128)        0         
_________________________________________________________________
dropout (Dropout)            (None, 25, 1, 128)        0         
_________________________________________________________________
flatten (Flatten)            (None, 3200)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               8

In [65]:
# import model checkpoint and early stopping
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [66]:
# intialize filepath, patience 
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
checkpointer = ModelCheckpoint(filepath='weights_base_model2d_5labels_v1.hdf5', verbose=1, save_best_only=True)

In [67]:
#compiling
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [68]:
#fit the model
model.fit(x_train, y_train, batch_size=16, epochs=100, 
          validation_data=(x_test,y_test), callbacks=[early_stopping, checkpointer])

Epoch 1/100
570/570 [==============================] - ETA: 0s - loss: 1.2593 - accuracy: 0.4497
Epoch 00001: val_loss improved from inf to 1.10740, saving model to weights_base_model2d_5labels_v1.hdf5
570/570 [==============================] - 33s 57ms/step - loss: 1.2593 - accuracy: 0.4497 - val_loss: 1.1074 - val_accuracy: 0.5167
Epoch 2/100
569/570 [============================>.] - ETA: 0s - loss: 1.0769 - accuracy: 0.5392
Epoch 00002: val_loss improved from 1.10740 to 1.08418, saving model to weights_base_model2d_5labels_v1.hdf5
570/570 [==============================] - 35s 61ms/step - loss: 1.0768 - accuracy: 0.5394 - val_loss: 1.0842 - val_accuracy: 0.5333
Epoch 3/100
570/570 [==============================] - ETA: 0s - loss: 1.0352 - accuracy: 0.5520
Epoch 00003: val_loss improved from 1.08418 to 1.06450, saving model to weights_base_model2d_5labels_v1.hdf5
570/570 [==============================] - 30s 52ms/step - loss: 1.0352 - accuracy: 0.5520 - val_loss: 1.0645 - val_accu

In [69]:
#train and test loss and scores respectively
train_loss_score = model.evaluate(x_train,y_train)
test_loss_score = model.evaluate(x_test,y_test)
print(train_loss_score)
print(test_loss_score)

15/15 [==============================] - 0s 14ms/step - loss: 0.9918 - accuracy: 0.6458
[0.43474090099334717, 0.865048885345459]
[0.9918153882026672, 0.6458333134651184]


# Base Model Performance

In [73]:
# base model perofrmance
preds1 = model.predict(x_test, batch_size=16, verbose=1)
preds1=preds1.argmax(axis=1)
preds1

30/30 [==============================] - 0s 12ms/step


array([2, 0, 4, 0, 0, 2, 2, 4, 4, 3, 3, 0, 3, 0, 4, 1, 1, 2, 2, 2, 0, 4,
       4, 1, 4, 2, 3, 4, 1, 2, 3, 4, 4, 1, 3, 0, 1, 1, 3, 0, 4, 0, 1, 1,
       2, 3, 3, 2, 4, 1, 0, 4, 2, 0, 4, 4, 4, 3, 2, 4, 4, 4, 4, 1, 1, 1,
       3, 0, 3, 2, 3, 4, 0, 1, 0, 3, 2, 0, 4, 0, 2, 4, 2, 0, 0, 4, 0, 3,
       2, 0, 2, 2, 2, 2, 4, 1, 4, 4, 3, 1, 0, 4, 3, 2, 2, 4, 3, 3, 3, 3,
       3, 2, 3, 1, 4, 3, 2, 3, 2, 4, 2, 4, 4, 0, 0, 3, 3, 1, 2, 3, 4, 2,
       2, 0, 4, 4, 3, 0, 2, 2, 0, 2, 3, 3, 0, 0, 3, 0, 3, 1, 3, 2, 4, 2,
       1, 0, 4, 2, 4, 0, 4, 2, 3, 3, 0, 0, 1, 3, 4, 0, 4, 1, 2, 4, 1, 0,
       2, 2, 4, 2, 0, 3, 0, 0, 0, 1, 2, 4, 0, 0, 2, 2, 4, 4, 3, 3, 3, 4,
       3, 4, 1, 2, 2, 2, 4, 1, 2, 0, 2, 1, 2, 1, 4, 4, 2, 3, 4, 2, 0, 3,
       3, 4, 0, 0, 2, 0, 0, 0, 1, 3, 1, 2, 0, 1, 4, 1, 2, 1, 3, 0, 0, 4,
       3, 3, 1, 4, 0, 3, 2, 3, 3, 2, 3, 0, 0, 2, 2, 3, 3, 4, 1, 3, 0, 1,
       3, 4, 3, 2, 4, 3, 4, 1, 2, 3, 3, 3, 4, 0, 4, 3, 2, 4, 3, 1, 0, 1,
       1, 2, 3, 4, 4, 4, 2, 4, 0, 2, 4, 0, 4, 3, 0,

In [74]:
# create a dataframe for predicted outputs and actual outputs
# predictions 
preds = preds1.astype(int).flatten()
preds = (lb.inverse_transform((preds)))
preds = pd.DataFrame({'predictedvalues': preds})

# Actual labels
actual=y_test.argmax(axis=1)
actual = actual.astype(int).flatten()
actual = (lb.inverse_transform((actual)))
actual = pd.DataFrame({'actualvalues': actual})

# Lets combined both of them into a single dataframe
finaldf = actual.join(preds)
finaldf[170:180]

,actualvalues,predictedvalues
170,Sad,Sad
171,Fear,Fear
172,Joy,Joy
173,Fear,Sad
174,Fear,Fear
175,Anger,Anger
176,Joy,Joy
177,Joy,Joy
178,Fear,Sad
179,None,Joy


In [75]:
# Write out the predictions to disk
finaldf.to_csv('Predictions.csv', index=False)
finaldf.groupby('predictedvalues').count()

,actualvalues
predictedvalues,
Anger,97
Fear,69
Joy,111
None,99
Sad,104


In [76]:
# Get the predictions file 
finaldf = pd.read_csv("Predictions.csv")
classes = finaldf.actualvalues.unique()
classes.sort()    

# Confusion matrix 
c = confusion_matrix(finaldf.actualvalues, finaldf.predictedvalues)
print("Model Accuracy:", accuracy_score(finaldf.actualvalues, finaldf.predictedvalues))

Model Accuracy: 0.6458333333333334


In [77]:
# Classification report 
classes = finaldf.actualvalues.unique()
classes.sort()    
print(classification_report(finaldf.actualvalues, finaldf.predictedvalues, target_names=classes))

              precision    recall  f1-score   support

       Anger       0.70      0.71      0.70        96
        Fear       0.67      0.48      0.56        96
         Joy       0.58      0.66      0.62        97
        None       0.65      0.67      0.66        95
         Sad       0.65      0.71      0.68        96

    accuracy                           0.65       480
   macro avg       0.65      0.65      0.64       480
weighted avg       0.65      0.65      0.64       480



In [78]:
# Save the model to disk
model_json = model.to_json()
with open("model_json_CNN2D_5labels_1.json", "w") as json_file:
    json_file.write(model_json)

In [79]:
from tensorflow.keras.models import load_model, model_from_json

In [3]:
# loading json and model architecture 
json_file = open('model_json_CNN2D_5labels_1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("weights_base_model2d_5labels_v1.hdf5")
print("Loaded model from disk")
 
# Keras optimiser
#opt = keras.optimizers.rmsprop(lr=0.00001, decay=1e-6)
loaded_model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
#score = loaded_model.evaluate(x_test, y_test, verbose=0)
#print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk


# Convert model to TFLite

In [4]:
# save tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
tflite_model = converter.convert()

# Save the TF Lite model.
with tf.io.gfile.GFile('model_5labels_v2_40f.tflite', 'wb') as f:
    f.write(tflite_model)

# TF-lite Performance

In [82]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="model_5labels_v2_40f.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [83]:
input_details

[{'name': 'conv2d_input',
  'index': 0,
  'shape': array([  1, 100,   1,   1], dtype=int32),
  'shape_signature': array([  1, 100,   1,   1], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [84]:
testData = np.expand_dims(x_test[4],axis=0)
atData = np.float32(testData)

In [85]:
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], atData)
print(input_data.dtype)
print(testData.shape)
interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

float32
(1, 100, 1, 1)
[[0.65234673 0.1016852  0.24128847 0.00315122 0.00152836]]


In [86]:
output_data.argmax(axis=1)

array([0])

In [87]:
y_test[4]

array([1., 0., 0., 0., 0.], dtype=float32)

In [88]:
preds = loaded_model.predict(x_test, batch_size=16, verbose=1)
preds=preds.argmax(axis=1)
preds

30/30 [==============================] - 0s 10ms/step


array([0, 0, 4, 0, 0, 2, 2, 4, 1, 4, 3, 2, 3, 0, 1, 2, 2, 2, 2, 0, 0, 1,
       4, 0, 4, 2, 3, 4, 2, 2, 3, 4, 4, 1, 3, 0, 2, 0, 3, 0, 4, 0, 4, 1,
       2, 4, 3, 2, 4, 1, 0, 4, 2, 0, 4, 4, 4, 3, 2, 1, 3, 1, 3, 1, 0, 1,
       3, 0, 3, 2, 0, 4, 0, 1, 0, 3, 0, 0, 4, 0, 2, 4, 2, 2, 3, 1, 0, 2,
       1, 0, 1, 2, 2, 1, 4, 1, 4, 4, 3, 1, 0, 3, 3, 0, 2, 4, 3, 3, 4, 3,
       0, 0, 3, 1, 4, 2, 2, 3, 2, 4, 2, 4, 4, 0, 0, 3, 4, 1, 2, 2, 4, 2,
       2, 0, 4, 4, 3, 0, 0, 3, 0, 2, 3, 3, 2, 0, 3, 0, 3, 1, 3, 0, 1, 1,
       4, 3, 4, 2, 1, 0, 4, 2, 3, 4, 2, 0, 3, 3, 4, 2, 4, 1, 2, 4, 1, 0,
       2, 2, 4, 1, 0, 3, 0, 0, 0, 1, 2, 4, 0, 0, 2, 2, 4, 3, 4, 3, 4, 4,
       3, 2, 1, 2, 4, 0, 4, 4, 2, 0, 2, 0, 0, 1, 1, 4, 2, 3, 1, 2, 0, 3,
       3, 4, 2, 0, 2, 0, 0, 0, 4, 3, 1, 2, 0, 1, 4, 4, 2, 1, 3, 0, 0, 4,
       2, 3, 0, 1, 2, 0, 4, 3, 3, 0, 3, 2, 0, 2, 3, 3, 3, 2, 1, 3, 0, 3,
       3, 3, 3, 2, 4, 3, 2, 1, 2, 3, 3, 3, 4, 2, 4, 3, 2, 4, 3, 1, 0, 1,
       1, 2, 0, 3, 1, 1, 2, 1, 0, 2, 0, 0, 4, 2, 0,

In [89]:
preds[4]

0

In [90]:
test_df.head()

,path,label
0,/home/bukya/Desktop/Speech_Emotion_Recognition...,Joy
1,/home/bukya/Desktop/Speech_Emotion_Recognition...,Joy
2,/home/bukya/Desktop/Speech_Emotion_Recognition...,Fear
3,/home/bukya/Desktop/Speech_Emotion_Recognition...,Anger
4,/home/bukya/Desktop/Speech_Emotion_Recognition...,Anger


In [91]:
test_df.tail()

,path,label
475,/home/bukya/Desktop/Speech_Emotion_Recognition...,None
476,/home/bukya/Desktop/Speech_Emotion_Recognition...,None
477,/home/bukya/Desktop/Speech_Emotion_Recognition...,Joy
478,/home/bukya/Desktop/Speech_Emotion_Recognition...,Fear
479,/home/bukya/Desktop/Speech_Emotion_Recognition...,None


In [79]:
#test_df.to_csv('test-set_5labels.csv', index=False)